**Autonomous Experimentation Setup -  Dropcasting Nanoparticles**

This notebook explains the process of how we set up the Autonomous Experimentation for Dropcasting and Analyzing the Nanoparticles in the Osuji's Lab. 


Before getting started, let's import all the pacakges that needed. 

Pyvisa - python pacakge enables communication between python programs and electronic instruments.                                
Instal pyvisa via: pip install -U pyvisa-py, in command prompt.

In [1]:
#Import Packages


from cgi import test
from optparse import Values
from sqlite3 import Row
from time import time
import tkinter as tk
from turtle import width
import pyvisa
import time 
# import keyboard
import time
import cv2
from PIL import Image, ImageTk
from webbrowser import get
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading as th
import minimalmodbus
from skopt import Optimizer
from skopt.plots import plot_objective
from skopt.plots import plot_evaluations
# from simple_pid import PID
import os

C:\Users\Osuji\AppData\Local\Temp\ipykernel_15160\3788331259.py:4: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  from cgi import test


Create a resource manager object, which is responsible for managing connections to and communication with various instruments connected to your computer and print all connected resouces.

In [2]:
rm = pyvisa.ResourceManager()

In [7]:
print(rm.list_resources()) 

('ASRL1::INSTR', 'ASRL6::INSTR', 'ASRL13::INSTR', 'ASRL18::INSTR')


Now establish a communication with a specific instrument connected to the port

In [6]:
#ASRL7 = Linear Stage
com_pxpy = rm.open_resource('ASRL7::INSTR')
com_pxpy.baud_rate = 921600 #fixed

#syringe pump
com_syr = rm.open_resource('ASRL4::INSTR')
# com_syr.baud_rate = 115200

#temperature control
#com_hot = minimalmodbus.Instrument('com17', 1, minimalmodbus.MODE_RTU)
#com_hot.serial.baudrate = 115200

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

**Setup the Motion Controller**

Instruct motion controller to enable motion for axis 1 and axis 2. Identify the live position of motion controller and print it.

In [25]:
#Turns on Motion Controller
com_pxpy.write('1MO')
com_pxpy.write('2MO')

#Identify current Motion Controller position and print
live_position_x_t = com_pxpy.query_ascii_values('1TP?')
live_position_x = ''.join(str(element) for element in live_position_x_t)
live_position_y_t = com_pxpy.query_ascii_values('2TP?')
live_position_y = ''.join(str(element) for element in live_position_y_t)
print('(x,y):')
print(live_position_x, live_position_y)


NameError: name 'com_pxpy' is not defined

Now define a function to command motion controller to move to a specified position

In [ ]:
def move_to(position_x, position_y):
    position_x = str(position_x)
    position_y = str(position_y)
    com_pxpy.write('1PA'+position_x)
    time.sleep(0.1)              
    com_pxpy.write('2PA'+position_y)
    time.sleep(0.1)

Defining a position where syringe alligns to the flush position and check by identifying the syringe position

In [ ]:
move_to(-40,-40)

In [ ]:
#Identifying the Syringe position
syringe_position_x = com_pxpy.query_ascii_values('1TP?')
syringe_position_x = ''.join(str(element) for element in syringe_position_x)
syringe_position_y = com_pxpy.query_ascii_values('2TP?')
syringe_position_y = ''.join(str(element) for element in syringe_position_y)
print(syringe_position_x,syringe_position_y)

**Setup the Syringe Pumping**

*First step is to initialize the syringe parameters*:

consider five syringe parameters: 

syringe diameter (mm) - we used hamiliton glass syringes 81520

reservior volume (ml) - usually ~5 or any integer

pump ID- pump number, the way we identified is bottom syringe is named as "0", and top syringe is named as "1"

infusion rate- pump rate in ul(microliter)/min (format: xx.x) -float, 1 decimal place

total volume to be dispensed- volume to be pumped at defined rate (format: xxx.x) - float, 1 decimal place

*Second step is to define a function to execute syring pumping*:

sends a series of commands to the syringe pump (com_syr.write()) to set up various parameters for pumping. After setting up the parameters, it sends a command to start the pumping procedure (irun command).

In [ ]:
#Initialize syringe parameters
sy_dia = 10.3 #mm 
res_vol = 4 #mL 
dispense_time = t_d = 30 #s
total_dispense_vol = V_d = 16 #uL
Q_d = 60 * (V_d/t_d) #mL/min
evaporation_time = e_t = 30 #=  18000 #s
sample_distance = 20 #mm #center-center distance of droplets

In [ ]:
#Defining an operation that executes the pumping procedure
def single_pump(sy_dia, res_vol, ID, rate, dispense_vol):
    com_syr.write(str(ID) + 'svolume ' + str(res_vol) + ' ml')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'diameter ' + str(float(sy_dia)))
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irate ' + str(float(rate)) + ' ul/min')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'tvolume ' + str(float(dispense_vol)) + ' ul')
    time.sleep(0.1)

    com_syr.write(str(ID) + 'irun')

**Setup the Initial Flushing**

Initial flushing fills the tubing with liquid. To set it up, the volume for flushing needs to be initialized first

Volume for flushing is calculated based on this:

Inner diameter of tubing is 1/32”

Radius = 1/64 Inch =  0.396875 mm

Volume of tubing = pi* R ** 2* L = 0.495 * L (mm ** 3)

V = N (mm ** 3) = N * 0.001 mL =  0.495 * L *  0.001 mL


In [ ]:
L_pipes = 400 #mm
V_flush_mL = 0.495 * L_pipes * 0.001 #mL #1/16 system
V_flush_uL = V_flush_mL * 1000 # Convert to uL

V_flush_initial = V_flush_uL
print(V_flush_initial)

Now define a function "flush()" to flush the piping system. It takes three parameters: flush_volume_0 (volume to flush from syringe 1), flush_volume_1 (volume to flush from syringe 2), and flush_time (time duration for flushing).





In [ ]:
def flush(flush_volume_0, flush_volume_1, flush_time): #flushes syringe 1 & 2
    #minki made flush_time = 3*flush_volume (around 20s)
    
    #calculation of rate of flush
    rate_flush_0 = 60 * (flush_volume_0 / flush_time)
    rate_flush_1 = 60 * (flush_volume_1 / flush_time)
    
    #flush the piping
    single_pump(sy_dia, res_vol, 0, rate_flush_0, flush_volume_0)
    time.sleep(0.1)

    single_pump(sy_dia, res_vol, 1, rate_flush_1, flush_volume_1)
    time.sleep(0.1)

Excecute the initial flushing through pipes

In [ ]:
#Initial flsh through pipes
move_to(syringe_position_x,syringe_position_y)
time.sleep(0.2)
flush(V_flush_initial, V_flush_initial, 10) #is time good?
time.sleep(0.2)

**Setup the Dropcast Loop**

This code is the core of dropcasting process, which includes depositing a droplet onto the substrate, waiting for evaporation, moving to the camera position for image analysis, and making any necessary optimizations or adjustments for the next set of parameters.

Droplet Volume and Flow rate is calculated based on this:

V = Q * t

	Volume to be dispensed = V_d
	Dispense time = t_d
	Dispense Rate = Q_d
	
	V_d = V_0 + V_1 

	V_d = Q_d *  t_d = (Q_0 + Q_1) * t_d

	TWO SYRINGES:

p = percentage of fluid one; (1-p) percentage of other fluid

	V_d = V_0 + V_1 = pV_d - (1-p)V_d
	V_0 = (p)V_d
	V_1 = (1-p)V_d 
"p" represents the concentration of NP, which is considered as an AE parameter

In [ ]:
#AE parameters 
file_path = "image"
p = [0.4,0.2]    

The dropcast loop iterates until all the droplets have been deposited

For each iteration:

1. It calculates the location of each droplet based on a predetermined pattern (in this case, two droplets per row)
2. It calculates the volume and infusion rate for each droplet based on predefined parameters and print the values
3. It moves the syringe to the location of the droplet and starts pumping the calculated volumes
4. It allows for some time for droplet evaporation
5. It moves the syringe to the camera location, captures an image, and performs image processing
6. It performs any necessary optimizations or adjustments

Finally loop ends after processing all droplets

Note: Flush setup can also be used in the loop to clear the piping for a fresh droplet, but in this instance, it is not necessary.

In [ ]:
# Dropcasting Loop
for i in range(len(p)):
    # Print current iteration
    print(f'iteration {i+1}')
    time.sleep(0.2)
    
    # Find location of each droplet in iteration
    n = 2  # number of droplets per row
    a, b = divmod(i, n)  # Calculate row and column indices
    print(a, b)
    
    #flush is to the right
    position_x = str(float(syringe_position_x)-(sample_distance*int(b)))
    position_y = str(float(syringe_position_y)-(sample_distance*int(a)))
    print(f'droplet center:{position_x,position_y}')
    
    # Calculate droplet volumes based on a predefined ratio
    V_0 = p[i] * V_d  # Volume for droplet 0
    V_1 = (1 - p[i]) * V_d  # Volume for droplet 1
    
    # Calculate infusion rates for each droplet
    Q_0 = 60 * (V_0 / t_d)  # Infusion rate for droplet 0
    Q_1 = 60 * (V_1 / t_d)  # Infusion rate for droplet 1
    
    print(f'droplet volume: {V_0, V_1}')
    
    # Move to the location of droplet i and start pumping
    move_to(position_x, position_y)
    single_pump(sy_dia, res_vol, 0, Q_0, V_0)  # Pump for droplet 0
    time.sleep(0.2)
    single_pump(sy_dia, res_vol, 1, Q_1, V_1)  # Pump for droplet 1
    time.sleep(0.2)
    
    # Allow droplet evaporation
    time.sleep(e_t)
    time.sleep(20)
    
    # Move to the camera location and capture an image
    #position_x = str(float(camera_position_x) - (sample_distance * int(b)) - flush_d)
    #position_y = str(float(camera_position_y) + (sample_distance * int(a)))
    #move_to(position_x, position_y)
    #time.sleep(0.2)
    #time.sleep(10)
    # Capture and save image
    #time.sleep(0.2)
    
    # Image processing
    
    # Optimization
    
    # End of loop
    print('end of loop')
